# 🗺️ Atelier Analyse de données géographiques

<a href="https://www.linkedin.com/company/data-for-good-grenoble/" target="_blank"><img src="https://github.com/data-for-good-grenoble/atelier-donnees-geographiques/blob/main/notebooks/D4G_logo.png?raw=1" width=200px/></a>

## 🗓️ Jeudi 15 mai 2024 - 18h45 - <a href="https://turbine.coop" target="_blank">La Turbine</a>

### PARTIE N°1 :

Pour commencer cet atelier, nous explorerons les packages d'analyse de données géographiques tels que Geopandas, Shapely ou Plotly sur des données en open data. Nous montrerons aussi comment télécharger des données d'OpenStreetMap 🗺️

### PARTIE N°2 :     

Nous ferons des mises en application sur des données de transports en commun et de sentiers de randonnée en Isère, pour lancer au passage notre prochain projet sur l'accessibilité des randos en bus 🚞💪

___

## PARTIE N°1

## ⚙️ Installation et import des bibliothèques

In [16]:
import geopandas as gpd
import shapely
import pandas as pd
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import Point, LineString, Polygon

## 📚 Liens vers la documentation des bibliothèques

### Manipulation de données

- <a href="https://pandas.pydata.org" target="_blank">Pandas</a>

Permet de manipuler et d’analyser des données tabulaires avec des structures comme les DataFrames.

### Géométrie et données géospatiales

- <a href="https://shapely.readthedocs.io/en/stable/index.html" target="_blank">Shapely</a>

Fournit des outils pour la manipulation de formes géométriques (points, lignes, polygones).

- <a href="https://geopandas.org/en/stable/" target="_blank">GeoPandas</a>

Étend Pandas pour gérer des données géographiques (GeoDataFrames) et facilite les opérations géospatiales.

### Visualisation de données

- <a href="https://matplotlib.org" target="_blank">Matplotlib</a>

Bibliothèque de base pour créer des graphiques statiques en 2D de manière personnalisable.

- <a href="https://plotly.com/python/" target="_blank">Plotly</a>

Permet de créer des visualisations interactives (grâce au Javascript), notamment des cartes et des graphiques 3D.

- <a href="https://python-visualization.github.io/folium/latest/" target="_blank">Folium</a>

Sert à créer des cartes interactives basées sur Leaflet.js, à partir de données Python.

## 🗂️ Sources et formats de données

- [🚌 Navettes saisonnières Transaltitude - Isère](https://transport.data.gouv.fr/datasets/desserte-des-stations-de-ski-iseroises-transaltitude-38)

## 🗺️ Visualisation de géométries avec Folium – Exemple sur Grenoble

Dans ce code, nous utilisons les bibliothèques Folium et Plotly pour créer une carte interactive centrée sur Grenoble. Nous y ajoutons ensuite différentes formes géométriques courantes utilisées en cartographie :

- 📍 Point
- ➖ Ligne (LineString)
- 🔲 Polygone

**Activité :**

Modifier la liste `polygon = [...]` afin que le polygone encadre précisément le parc Paul Mistral à Grenoble`

In [17]:
# Initialiser une carte centrée sur Grenoble
m = folium.Map(location=[45.188, 5.724], zoom_start=14)

# 1. Point
folium.Marker(location=[45.188, 5.724], popup="Point").add_to(m)

# 2. Ligne (LineString)
folium.PolyLine(
    locations=[[45.188, 5.724], [45.189, 5.726], [45.190, 5.728]],
    color='red',
    weight=2.5,
    popup="LineString"
).add_to(m)

# 3. Polygone
polygon = [
    [45.186, 5.722],
    [45.187, 5.725],
    [45.185, 5.727],
    [45.184, 5.723],
    [45.186, 5.722]
]
folium.Polygon(
    locations=polygon,
    color='purple',
    fill=True,
    fill_color='purple',
    popup="Polygone"
).add_to(m)

# Affichage ou sauvegarde
m

In [18]:
# 1. Point
point = Point(5.724, 45.188)

# 2. Ligne (LineString)
line = LineString([
    (5.724, 45.188),
    (5.726, 45.189),
    (5.728, 45.190)
])

# 3. Polygone
polygon = Polygon([
    (5.722, 45.186),
    (5.725, 45.187),
    (5.727, 45.185),
    (5.723, 45.184),
    (5.722, 45.186)
])

fig = go.Figure()

# 1. Point
fig.add_trace(go.Scattermapbox(
    lon=[point.x], lat=[point.y],
    mode='markers',
    marker=dict(size=10, color='blue'),
    name='Point'
))

# 2. Ligne (LineString)
fig.add_trace(go.Scattermapbox(
    lon=[coord[0] for coord in line.coords],
    lat=[coord[1] for coord in line.coords],
    mode='lines',
    line=dict(width=4, color='red'),
    name='LineString'
))

# 3. Polygone
fig.add_trace(go.Scattermapbox(
    lon=[coord[0] for coord in polygon.exterior.coords],
    lat=[coord[1] for coord in polygon.exterior.coords],
    mode='lines',
    fill='toself',
    fillcolor='rgba(128,0,128,0.3)',
    line=dict(color='purple'),
    name='Polygon'
))

# 3. Configuration de la carte
fig.update_layout(
    mapbox_style='open-street-map',
    mapbox_zoom=14,
    mapbox_center={'lat': 45.188, 'lon': 5.724},
    margin=dict(l=0, r=0, t=0, b=0)
)

fig.show()

## 🗺️ Récupération de données GeoJSON

Le projet Github france-geojson propose au format GeoJSON les cartes des **régions, départements, arrondissements, cantons et communes de France (métropole et départements d'outre-mer)** à partir des données publiées par l'<a href="https://www.ign.fr" target="_blank">IGN</a> et l'<a href="https://www.insee.fr/fr/accueil" target="_blank">INSEE</a>. Lien vers le site : https://france-geojson.gregoiredavid.fr

**Activité :**

- Récupérer l'url du fichier **GeoJSON** de l'**isère** sur le site france-geojson
- Avec <a href="https://geopandas.org/en/stable/" target="_blank">GeoPandas</a>, insérer les données dans un **GeoDataframe** (à l'aide de la fonction <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html" target="_blank">geopandas.read_file()</a>)
- Afficher ces données sur une **carte** (avec Matplotlib ou Folium) avec **toutes bonnes pratiques de cartographie** (à l'aide de la méthode <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html" target="_blank">geopandas.GeoDataFrame.plot()</a> ou de la méthode <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html" target="_blank">geopandas.GeoDataFrame.explore()</a> )

In [ ]:
# A COMPLETER

## 🌍 Choix du système de coordonnées de référence (CRS)

La propriété <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.area.html" target="_blank">geopandas.GeoSeries.area</a> permet de récupérer la surface des élèments d'une géométrie.

- Récupérer la **surface du département de l'isère**. Pourquoi cet avertissement ? (aide avec la propiété <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.crs.html" target="_blank">geopandas.GeoDataFrame.crs</a>)
- Changer de système de coordonnées de référence pour passer en **Web Mercator (EPSG:3857)** grâce à la méthode <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html" target="_blank">geopandas.GeoDataFrame.to_crs()</a>. Est-ce la bonne valeur de la surface ? Pourquoi ?
- Changer de système de coordonnées de référence pour passer en **RGF93 / Lambert-93 (EPSG:2154)**

In [ ]:
# A COMPLETER

## 🚌 Affichage des navettes saisonnières Transaltitude - Isère

Les **navettes hivernales desservant 14 stations de ski de l’Isère** peuvent être récupérées à cette adresse au format **GeoJSON** : https://transport.data.gouv.fr/datasets/desserte-des-stations-de-ski-iseroises-transaltitude-38

Comme précédemment :

- **Récupérer** les données dans un **GeoDataframe**
- **Afficher** les données sur une **carte**

In [9]:
# A COMPLETER

## Récupération de données depuis Open Street Map avec Overpass Turbo

## PARTIE N°2

